**三种常用分词方法**

In [ ]:
'''1.基于字典（基于字符串匹配）'''
#比如最大正向匹配
class MMSeg():
    
    def __init__(self):
        self.segment_dict = {"伟大", "中国", "勤劳","中国人民", "北京", "人民政府"}#分词词典，一般比较大
        self.hot_dict = {"伟大", "中国", "中国人民", "北京", "人民政府"}#热词词表
        self.max_word_length = 4#词语的最大长度，可调
        
    def segment(self, text):
        index  = 0#存储当前索引
        words = []#存储分词结果
        while index<len(text):
            #每一轮匹配，就是在一个窗口内，遍历所有前缀紫子字符串
            #首先计算窗口最右端的索引。快到文本末尾时，需要截断。
            window_end_index = min(len(text), index + self.max_word_length)
            #传进来的是指针，如果在text末尾添加3个符号，会改变text;深拷贝一个的话，还需要操作内存，消耗也挺大。
            word = None#用于存储本轮匹配到的词语
            for i in range(window_end_index, index, -1):#由长到短匹配
                sub_string = text[index:i]
                if sub_string in self.segment_dict:
                    word = sub_string
                    break#如果匹配到合适的词语，更短的就不需要考虑了
            if word==None:#如果没有匹配到词语，index对应的字单独成词，索引加一
                words.append(text[index]) 
                index += 1
            else:#如果匹配到词语，收集这个词语，索引加word的长度
                words.append(word) 
                index += len(word) 
        return words 
    
    def find_hot_words(self, text):
        index  = 0#存储当前索引
        hot_words = []#存储识别出的热词
        while index<len(text):
            #每一轮匹配，就是在一个窗口内，遍历所有前缀紫子字符串
            #首先计算窗口最右端的索引。快到文本末尾时，需要截断。
            window_end_index = min(len(text), index + self.max_word_length)
            
            hot_word = None#用于存储本轮匹配到的词语
            for i in range(window_end_index, index, -1):#由长到短匹配
                sub_string = text[index:i]
                if sub_string in self.hot_dict:
                    hot_word = sub_string
                    break#如果匹配到合适的词语，更短的就不需要考虑了
            if hot_word==None:#如果没有匹配到词语，索引加一
                index += 1
            else:#如果匹配到热词，收集这个词语，索引加word的长度
                hot_words.append(hot_word) 
                index += len(hot_word) 
        return hot_words
    
if __name__ == '__main__':
    seg = MMSeg()
    s  ="伟大的中国人民是勤劳的"
#     s = "在中国人民政府是很重要的"
    print(seg.segment(s))#打印分词结果
    print(seg.find_hot_words(s))#打印热词识别结果  
    
    
    
    
    
corpus=["我","今天","特别","特别想","非常","非常想","吃","芒果"]
sentence="我今天特别想吃芒果啊"
max_num=3#最大词条长度，这里一眼就看出来，对于大的语料词典，用下边计算
max_num=0
for i in corpus:
    if len(i)>max_num:
        max_num=len(i)
def word_segment(corpus,sentence,max_num):
    start=0
    result=[]
    flag=0
    while start<len(sentence):
        window_end=min(len(sentence),start+max_num)#滑动窗口
        word=None
        for i in range(window_end,0,-1):
            sub_string=sentence[start:window_end]
            if sub_string in corpus:
                word=sub_string
                break
        if word:
            result.append(word)
            start+=len(word)
        else:
            result.append(sentence[start])
            start+=1
    return result

word_segment(corpus,sentence,max_num)              

In [ ]:
'''基于统计
1.找出句子的所有分词结果
2.在所有结果里找到最好的一个，用语言模型也可以'''
from collections import defaultdict
#针对步骤一：类似于LeetCode140，单词匹配II那个，返回所有匹配的结果
def wordBreak(s,wordDict,dic):
    if not s:
        return []
    if s in dic:
        return dic[s]
    res=[]
    for word in wordDict:
        n=len(word)
        if s[:n]==word:
            for r in wordBreak(s[n:],wordDict,dic):
                if r:
                    res.append(word+" "+r)

    return res
    
corpus={"我们","我","今天","特别","特别想","非常","非常想","吃","芒果","想"}
sentence="我今天特别想吃芒果"
print(wordBreak(sentence,corpus,{}))
#步骤二：找最好的分词结果，类似于这个
'''文本分词，基于词典，计算最大分数的切分组合：
https://www.cnblogs.com/naniJser/p/6058775.html
类似于jieba的DAG ，
从后往前，动态规划问题：动态转移方程：route[idx] = max((log(dict.FREQ.get(sentence[idx:x + 1]) or 1) -
                              logtotal + route[x + 1][0], x) for x in DAG[idx])'''

DAG={0: [0], 1: [1, 2, 4], 2: [2], 3: [3, 4], 4: [4]}
s="去北京大学"
word_scores={"北京大学":100,"北京大":0,"大学":50,"去":100,"玩":100,"北京":20,"北":1,"京":1,"大":1,"学":1}
def findMaxScorepath(s,DAG,word_scores):
    n=len(s)
    dp=[0]*(n+1)
    #从后往前计算
    for i in range(n-1,-1,-1):
        #word_scores.get(s[i:x+1],0)表示如果该分词方式不在字典里，则为0
        dp[i]=max([word_scores.get(s[i:x+1],0)+dp[x+1] for x in DAG[i]])
    return dp[0]
findMaxScorepath(s,DAG,word_scores)
#[100+100=200]

#HMM分词


**DAG**

In [ ]:
'''有向无环图构建：然后基于前缀词典，对输入文本进行切分，对于“去”，没有前缀，那么就只有一种划分方式；对于“北”，
则有“北”、“北京”、“北京大学”三种划分方式；对于“京”，也只有一种划分方式；对于“大”，
则有“大”、“大学”两种划分方式，依次类推，可以得到每个字开始的前缀词的划分方式。'''
def get_DAG(sentence):
    global FREQ
    DAG={}
    N=len(sentence)
    for k in range(N):
        tmplist=[]
        i=k
         # 位置k形成的片段
        frag=sentence[k]
        #如果该字或词在字典里边:
        while i<N and frag in FREQ:
            # 如果该片段的词频大于0
            # 将该片段加入到有向无环图中
            # 否则，继续循环
            if FREQ[frag]:
                tmplist.append(i)#自己的位置
            i+=1
            # 新的片段较旧的片段右边新增一个字
            frag=sentence[k:i+1] 
#             print(frag)
        #生成这个词的有向无环图
        DAG[k] = tmplist
    return DAG
    #简单构建一个词和相应词频字典
FREQ={"北京大学":2053,"北京大":0,"大学":20025,"去":12340,"玩":4207,"北京":34488,"北":17860,"京":6583,"大":144099,"学":17482}
sentence="去北京大学"
DAG=get_DAG(sentence)
DAG       

In [ ]:
'''动态规划：因为汉语句子的重心经常落在后面, 就是落在右边, 因为通常情况下形容词太多,
后面的才是主干, 因此, 从右往左计算, 正确率要高于从左往右计算, 这个类似于逆向最大匹配！'''
from math import log
total=sum(FREQ.values(),0)
def calc(sentence,DAG):  #动态规划，计算最大概率的切分组合
	#输入sentence是句子，DAG句子的有向无环图
    N = len(sentence)  #句子长度
    min_freq=1
    logtotal=log(total)
    route={}
    route[N] = (0.0,'')
    for idx in range(N-1,-1,-1):  #和range用法一样，不过还是建议使用xrange
		#可以看出是从后往前遍历每个分词方式的
	   #下面的FREQ保存的是每个词在dict中的频度得分，打分的公式是 log(float(v)/total)，其中v就是被打分词语的频数
		 #FREQ.get(sentence[idx:x+1,min_freq)表示，如果字典get没有找到这个key，那么我们就使用最后的frequency来做
		 #由于DAG中是以字典+list的结构存储的，所以确定了idx为key之外，
		 #仍然需要for x in DAG[idx]来遍历所有的单词结合方式（因为存在不同的结合方法，例如“国”，“国家”等）
		 #以（频度得分值，词语最后一个字的位置）这样的tuple保存在route中
        candidates = [ (log(FREQ.get(sentence[idx:x+1]) or min_freq)-logtotal+ route[x+1][0],x) for x in DAG[idx] ]
        route[idx] = max(candidates)
    return route[0]
calc(sentence,DAG)
